# Instructions

The following code was designed in order to implement batch processing of location tracking.  Currently, for this to work, all videos must take the same settings, including the defined regions of interest.  To confirm that regions of interest are consistent across videos, for each video an overlay of the reference frame, the regions of interest and the animal trace are produced.  Moreover, the reference frame is currently required to be generated by taking the median of each individual video.  In addition to saving frame by frame location, distance travelled, and whether the animal is in each ROI in separate csv files, bins can also be defined for summarizing videos (e.g. minute by minute).  All summary information will be saved in a single file.

# 1. Load Necessary Packages
The following code loads neccessary packages and need not be changed by the user.

In [ ]:
%load_ext autoreload
%autoreload 2
import os
import holoviews as hv
import numpy as np
import pandas as pd
import LocationTracking_Functions as lt

# 2. User Defines Batch Processing Directory and Settings
Below, options are set by user for batch processing of videos.  If you are unfamiliar with these settings, please see LocationTracking_Individual.ipynb.

In [ ]:
#define video paramaters
video_dict = {
    'dpath'        : "/Users/DeniseCaiLab/Videos",  
    'ftype'        : 'mp4',
    'start'        : 0, 
    'end'          : None,
    'region_names' : None, #['Left','Right']
    'dsmpl'        : 1,
    'stretch'      : dict(width=1, height=1)
}

#define parameters for location tracking
tracking_params = {
    'loc_thresh'    : 99, 
    'use_window'    : True, 
    'window_size'   : 100, 
    'window_weight' : .9, 
    'method'        : 'abs',
    'rmv_wire'      : False, 
    'wire_krn'      : 10
}

#set bin_dict
#set bin_dict = None if only overall session average is desired
bin_dict = {
    '1' : (0,10),
    '2' : (10,20),
    '3' : (20,30)
}


#code below loads folder with files.  
video_dict = lt.Batch_LoadFiles(video_dict)
video_dict['FileNames']

# 3. (Optional) Crop Image if Desired
To crop video frame, after running code below, select box selection tool below image (square with a plus sign).  To start drawing region to be included in analyis, double click image.  Double click again to finalize region.  If you decide to change region, it is best to rerun this cell and subsequent steps.  Note that this is done based upon first video in folder.

In [ ]:
%%output size=100 

img_crp, video_dict = lt.LoadAndCrop(video_dict, cropmethod='Box', fstfile=True)
img_crp

# 4. (Optional) Mask Internal Regions

The following code is used to exclude internal portion of image from the field of view. After running cell below, draw regions to be excluded.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell. 

In [ ]:
%%output size=100

img_mask, video_dict['mask'] = lt.Mask_select(video_dict, fstfile=True)
img_mask

# 5. (Optional) Define Regions of Interest

After running cell below, draw regions of interest on presented image in the order you provided them.  To start drawing a region, double click on image.  Single click to add a vertex.  Double click to close polygon.  If you mess up it's easiest to re-run cell.  Note that this is done based upon first video in folder.

In [ ]:
%%output size=100

video_dict['reference'], img_ref = lt.Reference(video_dict, fstfile=True, num_frames=50) 
img_roi, video_dict['roi_stream'] = lt.ROI_plot(video_dict)
img_roi

---
# 6. (Optional) Define Scale for Distance Calculations

### 6a. Select two points of known distance

After running cell below, click on any two points and the distance between them, in pixel units, will be presented/returned. Will be used to convert pixel distance to other scale. Note that once drawn, points can be dragged or you can click again.

In [ ]:
%%output size = 100

video_dict['reference'], img_ref = lt.Reference(video_dict, fstfile=True, num_frames=100) 
img_scl, video_dict['scale'] = lt.DistanceTool(video_dict)
img_scl

### 6b. Define real-world distance between points
Below, set the distance between the points selected, and the scale. Note that scale can be any desired text.

In [ ]:
distance = 100
scale = 'cm'

video_dict['scale'] = lt.setScale(distance, scale, video_dict['scale'])

# 7. Perform Batch Processing and Display Traces from Each Session

The code below will save frame by frame data for each video file in its own csv. Binned summary information will be saved in a single file entitled 'BatchSummary.csv'. Additionally, the reference frame for each session will be displayed, along with a trace of the regions of interest (if supplied), and a trace of the animal's location across the session.

In [ ]:
%%opts Layout [shared_axes=False] 
%%output size=100 

summary, images = lt.Batch_Process(video_dict,tracking_params,bin_dict)
images.cols(2)